<a href="https://colab.research.google.com/github/janak11111/NLP_Project/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install libraries
import nltk
!pip install sentence-transformers
!pip install autocorrect
!pip install -U spacy
!python -m spacy download en_core_web_sm
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
# spacy.cli.download("en_core_web_sm")
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 5.8 MB 25.7 MB/s 
     |████████████████████████████████| 1.3 MB 48.2 MB/s 
     |████████████████████████████████| 182 kB 59.3 MB/s 
     |████████████████████████████████| 7.6 MB 37.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1fbfb2fc1589b2ec6085e6fc851ba297b8610d93bc654ae77be8814c79588485
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 622 kB 5.2 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=a767ab03676aeddbc1a733ab5fa60698

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# import libraries
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import spacy
import re
nlp = spacy.load('en_core_web_sm')

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [ ]:
# read dataset
data = pd.read_csv("F2_Claim_Check_Worthiness_train.csv")

In [ ]:
# preprocessing funciton created
def preprocessing(df):
    df1 = df.copy()
    df1['white_space_removed'] = 0
    df1['tokenized_data'] = 0
    df1['stopword_removed_data'] = 0
    df1['punct_removed_data'] = 0
    df1['url_removed_data'] = 0
    df1['spelling_checked_data'] = 0
    stop_words = set(stopwords.words("english"))

    # create object for lemmetizer and spelling checking
    # lemmatizer = WordNetLemmatizer()
    spell = Speller(lang='en')


    # iterate over each row of dataset and preprocess data
    for i in range(df1.shape[0]):

        # white space removel
        df1['white_space_removed'][i] = re.sub("\s+", " ", df1.tweet[i])


        # lower casing and tokenization
        lower = df1['white_space_removed'][i].lower()
        tokenized_data = word_tokenize(lower)
        df1['tokenized_data'][i] = tokenized_data
        # print(tokenized_data)


        # remove stopwords
        stopword_removed_data = [x for x in tokenized_data if x not in stop_words]
        df1['stopword_removed_data'][i] = stopword_removed_data 
        # print(stopword_removed_data)


        # punctuation removel
        punct_removed_data = [x for x in stopword_removed_data if x.isalnum()]
        df1['punct_removed_data'][i] = punct_removed_data
        # print(punct_removed_data)


        # remove urls and html tags
        urls = re.findall("https?://[a-zA-Z0-9_\?=\@\/#=.~-]+", " ".join(punct_removed_data))
        url_removed_data = [x for x in punct_removed_data if x not in urls]
        df1['url_removed_data'][i] = url_removed_data
        # print(url_removed_data)
        # print(type(url_removed_data))


        # spelling checking
        spelling_checked_data = [spell(x) for x in url_removed_data]
        df1['spelling_checked_data'][i] = " ".join(spelling_checked_data)
        # print(" ".join(spelling_checked_data))

    return df1

In [ ]:
df = preprocessing(data)

In [ ]:
# rename dataframe
df = df.rename({'spelling_checked_data': 'preprocessed_text'}, axis=1)
df.head()

In [ ]:
# extract required feature
df_preprocessed = df[['tweet','label','id','preprocessed_text']]

In [ ]:
# store preprocessed data
df_preprocessed.to_csv("preprocessed_train.csv", index=False)

In [ ]:
# read train data
df_preprocessed_train = pd.read_csv("preprocessed_train.csv", index_col=False)
df_preprocessed_train.head()

,tweet,label,id,preprocessed_text
0,"India's gift of 100,000 COVID-19 vaccines arri...",0,0,india gift vaccines arrived barbados earlier t...
1,As part of the ongoing nationwide vaccination ...,0,1,part ongoing nationwide vaccination rollout se...
2,"Pleased to receive 50,000 doses of Covid-19 va...",0,2,pleased receive doses vaccines india morning g...
3,Four former presidents have banded together fo...,0,3,four former presidents banded together two nat...
4,WSJ: All three of Russia's main intelligence s...,1,4,wsj three russia main intelligence services sr...


In [ ]:
# # read test data and store preprocessed test data
df_test=pd.read_csv("F2_Claim_Check_Worthiness_test.csv")
df_test_preprocess = preprocessing(df_test)

df_test_preprocess = df_test_preprocess.rename({'spelling_checked_data': 'preprocessed_text'}, axis=1)
df_test_preprocess_1 = df_test_preprocess[['tweet','id','preprocessed_text']]
df_test_preprocess_1.to_csv('preprocessed_test.csv')

In [ ]:
# read test data
df_preprocessed_test = pd.read_csv('preprocessed_test.csv')
df_preprocessed_test.head()

,Unnamed: 0,tweet,id,preprocessed_text
0,0,Native American communities have been hit hard...,0,native american communities hit hard thanks in...
1,1,ALERT: CVS Pharmacy is now offering COVID-19 v...,1,alert cvs pharmacy offering vaccines massachus...
2,2,#COVID19 vaccines are an important tool to hel...,2,covid19 vaccines important tool help stop pand...
3,3,Pfizer reports that IRL the vaccine is 97% eff...,3,prize reports irl vaccine 97 effective symptom...
4,4,Vaccines are complex medicines. Europeans can ...,4,vaccines complex medicines european trust vacc...


In [ ]:
# extract pos tag feature
def pos_tag_extraction(sentence):
    docs = nlp(sentence)
    pos_tags_list=[]
    for token in docs:
        pos_tags_list.append(token.pos_)
    pos_tags = " ".join(pos_tags_list)
    return pos_tags

In [ ]:
# extract dependency feature
def dep_feature_extraction(sentence):
    docs = nlp(sentence)
    dep_list=[]
    for token in docs:
        dep_list.append(token.dep_)
    dep_feature = " ".join(dep_list)
    return dep_feature

In [ ]:
# find pos tag vector of sentences
def get_pos_vector(pos_sentence):
    pos_tags = {'ADJ':0,'ADV':0,'NOUN':0,'PROPN':0,'VERB':0,'NUM':0,'ADP':0,'PRON':0} 
    
    for pos in pos_sentence.split():        
        if pos in pos_tags:
            pos_tags[pos] += 1
            
    return list(pos_tags.values())

In [ ]:
# create list of all unique dependencies in entire data
def get_count_pos_dep_dict(df):
    unique_dep = set()
    for row in df['preprocessed_text']:
        pos_sentence = pos_tag_extraction(row).split()
        dep_sentence = dep_feature_extraction(row).split()
        for pos, dep in zip(pos_sentence, dep_sentence):
            unique_dep.add(pos+"_"+dep)
    return list(unique_dep)

dependency_list = get_count_pos_dep_dict(df_preprocessed_train)

In [ ]:
# occurance count of all unique dependencies in entire data    
def get_pos_dep_dict(df):
    unique_dep = {}
    for row in df['preprocessed_text']:
        pos_sentence = pos_tag_extraction(row).split()
        dep_sentence = dep_feature_extraction(row).split()
        for pos, dep in zip(pos_sentence, dep_sentence):
            if (pos+"_"+dep) not in unique_dep:
                unique_dep[(pos+"_"+dep)] = 1
            else:
                unique_dep[(pos+"_"+dep)] += 1            
    return unique_dep

dependency_dict = get_pos_dep_dict(df_preprocessed_train)

In [ ]:
# find most important dependency
from collections import Counter

common_dep = Counter(dependency_dict).most_common(15)
important_dep = []

for dep in common_dep:
    important_dep.append(dep[0])
print(important_dep)

['NOUN_compound', 'ADJ_amod', 'NOUN_dobj', 'PROPN_compound', 'NOUN_nsubj', 'VERB_ROOT', 'VERB_ccomp', 'NUM_nummod', 'ADV_advmod', 'PROPN_nsubj', 'VERB_acl', 'NOUN_nmod', 'NOUN_npadvmod', 'VERB_amod', 'VERB_advcl']


In [ ]:
# check lengt of most important dependency
len(important_dep)

15

In [ ]:
# get dependency vector of depencency sentence
def get_dep_vector(dep_sentence): 
    pos_tags = {'ADJ':0,'ADV':0,'NOUN':0,'PROPN':0,'NUM':0,} 
    pos_sentence = pos_tag_extraction(dep_sentence).split(" ")
    dep_sentence = dep_feature_extraction(dep_sentence).split(' ')
    sentence_dep_dict = dict.fromkeys(important_dep, 0)
    for pos, dep in zip(pos_sentence, dep_sentence):
      if pos in pos_tags:
        if (pos+"_"+dep) in sentence_dep_dict and (pos+"_"+dep) in important_dep:
            sentence_dep_dict[(pos+"_"+dep)] += 1
    return list(sentence_dep_dict.values())

In [ ]:
# scale the dataframe
def scale_df(df):
  scaler = MinMaxScaler()
  temp = df.copy()
  transformed_data = pd.DataFrame(scaler.fit_transform(temp))
  return transformed_data

In [ ]:
# apply pca on data
def apply_pca(df, component=190):
  pca = PCA(n_components=component)
  pca_df = pd.DataFrame(pca.fit_transform(df))
  print("variance captuered is :", sum(pca.explained_variance_ratio_))
  return pca_df

In [ ]:
# find sentence embedding using sentence transformer
def get_sentence_embedding(df):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  embeddings = []
  for i in range(len(df)):
    embeddings.append(model.encode(df['tweet'][i]))
  sentence_embedding = pd.DataFrame(embeddings)
  return sentence_embedding

In [ ]:
# create pipeline for all computation
def pipeline(df):
  # get pos and dep feature
  final_feature = []
  for row in df['preprocessed_text']:
      temp = []
      temp.extend(get_pos_vector(row))
      temp.extend(get_dep_vector(row))
      final_feature.append(temp)
  df_features = pd.DataFrame(final_feature)

  # sentence embeddings
  sentence_embedding = get_sentence_embedding(df)
  print(sentence_embedding.shape)
  
  # merge data
  final_features = pd.concat([df_features, sentence_embedding], axis=1)
  final_features.columns = [list(range(len(df_features.columns) + len(sentence_embedding.columns)))]

  # scale data
  scaled_df = scale_df(final_features)

  # apply PCA
  pca_df = apply_pca(scaled_df)

  # return dataframe
  return pca_df

In [ ]:
# apply pipeline on preprocessed train data
final_df_train = pipeline(df_preprocessed_train)
final_df_train.head()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

(2122, 384)
variance captuered is : 0.9273593835154551


,0,1,2,3,4,5,6,7,8,9,...,180,181,182,183,184,185,186,187,188,189
0,0.206153,-0.158735,-0.849912,-0.282595,-0.292050,0.287748,-0.871215,-0.318030,-0.143684,-0.715494,...,-0.020627,0.051865,0.016993,-0.011362,-0.095037,-0.041242,-0.010619,0.068452,0.050942,0.064868
1,-0.360402,-0.055132,-0.157903,0.590786,0.202913,-0.023279,-0.173956,0.236162,0.513071,-0.323092,...,0.011741,-0.013883,0.162921,-0.044280,0.146323,0.084247,-0.096035,0.069144,0.098395,0.185388
2,0.132690,-0.612062,-0.927085,0.263096,-0.212938,0.096086,-0.638951,-0.078035,0.186291,-0.653751,...,0.072826,-0.046035,-0.086896,-0.018909,-0.059418,-0.105645,-0.134283,0.065729,0.056750,0.146596
3,0.233983,0.367775,0.085643,-0.427676,-0.322165,0.037932,-0.019876,0.767667,0.080845,-0.065849,...,-0.178480,-0.002605,0.085450,0.007876,-0.002962,-0.083570,0.034154,0.093986,0.144944,-0.197623
4,0.944782,0.046538,0.696557,-0.683370,0.691449,0.860789,0.254227,0.416152,1.199216,-0.070273,...,-0.007781,0.067807,0.052104,0.136330,0.032839,-0.035339,-0.148597,0.056701,0.022083,0.126433


In [ ]:
# apply pipeline on preprocessed test data
final_df_test = pipeline(df_preprocessed_test)
final_df_test.head()

(195, 384)
variance captuered is : 0.9995523898010144


,0,1,2,3,4,5,6,7,8,9,...,180,181,182,183,184,185,186,187,188,189
0,-1.084414,0.106659,-0.627027,-0.231803,0.513259,0.443088,0.096683,-0.158113,-0.458001,1.339746,...,0.115732,0.059663,0.053048,-0.021777,0.013482,-0.072548,-0.078235,-0.025441,0.077437,0.043365
1,-0.838130,0.779988,0.380810,-0.398866,-0.800270,-0.118242,-0.453681,0.086177,0.247269,-0.469840,...,0.006219,-0.030068,-0.005755,-0.075828,-0.001851,-0.047542,0.061757,0.010910,0.062238,0.042447
2,-0.892136,-0.256970,-0.798804,-0.038894,0.012618,-0.136972,1.118259,0.089635,-0.585645,0.306159,...,-0.073781,0.077143,0.070729,-0.025324,0.036158,-0.043235,0.069382,-0.092380,-0.046064,0.012170
3,-0.415014,-0.942185,0.359460,1.031324,0.481922,-0.488846,-1.130063,0.004571,-0.328225,0.055591,...,0.014987,0.013202,-0.000270,-0.006008,-0.000008,0.000865,-0.006788,-0.008719,-0.020514,0.028702
4,-0.495199,-1.661914,0.561369,-0.220856,0.120079,-0.016299,-0.221242,-0.396409,0.882624,-0.321744,...,0.010582,0.030535,-0.031951,0.017181,-0.153795,-0.068028,0.038158,-0.089674,0.008551,0.004220


In [ ]:
# count of labels
print("distribution of label in train dataset")
df_preprocessed_train.label.value_counts()

distribution of label in train dataset


0    1675
1     447
Name: label, dtype: int64

In [ ]:
# split te data into train and validation
y = df_preprocessed_train.label
X_train, X_test, y_train, y_test = train_test_split(final_df_train, y, train_size=0.7, random_state=42, stratify=y)

In [ ]:
# train data on SVM model
model = SVC(C=1, gamma=0.08, class_weight='balanced')
model.fit(X_train, y_train)

# fit data on model
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)


# test data on model
print("train macro f1 score:", f1_score(y_train, y_pred_train, average='macro'))
print("test macro f1 score:", f1_score(y_test, y_pred_test, average='macro'))

train macro f1 score: 0.8595027247956404
test macro f1 score: 0.7227251802976831


In [ ]:
# compute test labels
y_pred = model.predict(final_df_test)
len(y_pred)

195

In [ ]:
# save test labels with id
ID = [i for i in range(len(y_pred))]
output_df = pd.DataFrame(list(zip(y_pred, ID)), columns=['label', 'id'])
output_df.to_csv("output5.csv", index=False)

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred_test))

[[396 107]
 [ 32 102]]


In [ ]:
# classification report
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.93      0.79      0.85       503
           1       0.49      0.76      0.59       134

    accuracy                           0.78       637
   macro avg       0.71      0.77      0.72       637
weighted avg       0.83      0.78      0.80       637

